## Imports:

In [ ]:
import os
import librosa
import numpy as np

## Data Loading and Preprocessing:

In [ ]:
def extract_features(file_path, max_pad_len=174):
    audio, sr = librosa.load(file_path, res_type="kaiser_fast")
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode="constant")
    
    return mfccs

def prepare_dataset(dataset_path):
    X = []
    y = []
    for folder in os.listdir(dataset_path):
        for file in os.listdir(os.path.join(dataset_path, folder)):
            file_path = os.path.join(dataset_path, folder, file)
            mfccs = extract_features(file_path)
            X.append(mfccs)
            y.append(folder)

    return np.array(X), np.array(y)